### 212. Word Search II

$r$ is the number of rows.  
$c$ is the number of columns.  
$t$ is the maximum length of any word in the array words.  
$s$ is the sum of the lengths of all the words. 

#### Backtracking (Trie + Hash Set)

**時間複雜度: $O(r * c * 4 * 3^{t-1} + s)$**  
**空間複雜度: $O(s)$**

In [1]:
from typing import List

class TrieNode():
    def __init__(self):
        self.children = {}
        self.end = False
    def add_word(self, word):
        current = self
        for char in word: # time: O(t)
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]

        current.end = True

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        root = TrieNode()
        for word in words: # time: O(s)，最深為單詞長度 t，但相較於總節點數 s 通常可忽略
            root.add_word(word)

        row_length = len(board)
        col_length = len(board[0])
            
        result = set()
        visit = set()

        def dfs(row, col, node, word):
            if (
                row < 0 or col < 0 or
                row >= row_length or col >= col_length or
                (row, col) in visit or
                board[row][col] not in node.children
            ):
                return
            
            visit.add((row, col))
            node = node.children[board[row][col]]
            word += board[row][col]
            if node.end:
                result.add(word)

            # time: O(4)，第一次可以走上下左右 4 個方向
            # time: O(3^(t-1))，後續不能回到前一次的位置，只有三個方向
            dfs(row-1, col, node, word)
            dfs(row+1, col, node, word)
            dfs(row, col-1, node, word)
            dfs(row, col+1, node, word)

            visit.remove((row, col))

        for row in range(row_length): # time: O(r)
            for col in range(col_length): # time: O(c)
                dfs(row, col, root, '')

        return list(result)

In [2]:
board = [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]]
words = ["oa","oaa"]
# Output: ["eat","oath"]

Solution().findWords(board, words)

['oa', 'oaa']

#### Backtracking

**時間複雜度: $O(r * c * 4^t + s) = O(r * c * 4^t)$**  
**空間複雜度: $O(t)$**

In [3]:
from typing import List

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 取得棋盤的行數與列數
        row_length = len(board)
        col_length = len(board[0])
        # 儲存最終找到的單詞
        result_list = []

        # 定義 DFS 函式，用來檢查從 (row, col) 開始能否找到 word[index:]
        def dfs(word, index, row, col):
            # 如果已經走到單詞的最後一個字母之後，代表整個單詞找到
            if index == len(word):
                return True
            
            # 邊界檢查 & 是否匹配字母 & 是否已訪問過
            if (
                row < 0 or col < 0 or
                row >= row_length or col >= col_length or
                word[index] != board[row][col] or
                board[row][col] == '#'
            ):
                return False
            
            # 暫時標記為已訪問
            board[row][col] = '#'

            # 往上下左右四個方向繼續搜尋下一個字母 # time: O(4^t)
            result_flag = (
                dfs(word, index+1, row-1, col) or
                dfs(word, index+1, row+1, col) or
                dfs(word, index+1, row, col-1) or
                dfs(word, index+1, row, col+1)
            )

            # 回溯：恢復原本的字母，讓其他搜尋路徑可以使用
            board[row][col] = word[index]

            return result_flag

        # 針對每個單詞進行搜尋
        for word in words: #time: O(s)，+s 是在遍歷 words 陣列、讀取字元的過程中累積的總成本，不是 DFS 的搜尋成本
            flag = False  # 紀錄該單詞是否已經找到
            for row in range(row_length): # time: O(r)，以下才是 DFS 的成本
                if flag:  # 如果已經找到，提前跳出
                    break
                for col in range(col_length): # time: O(c)
                    # 如果首字母不符合，直接跳過
                    if board[row][col] != word[0]:
                        continue
                    # 如果能找到該單詞，加入結果
                    if dfs(word, 0, row, col):
                        result_list.append(word)
                        flag = True
                        break

        return result_list

In [4]:
board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain"]
# Output: ["eat","oath"]

Solution().findWords(board, words)

['oath', 'eat']